# Rolling Horizon

Solve large operational problems by decomposing the time horizon into sequential segments.

This notebook introduces:

- **Rolling horizon optimization**: Divide time into overlapping segments
- **State transfer**: Pass storage states and flow history between segments
- **When to use**: Memory limits, operational planning with limited foresight

## Setup

In [ ]:
import timeit

import numpy as np
import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

## Create a Test System

We'll use a simple heating system with storage to demonstrate the rolling horizon approach:

In [ ]:
# One week at hourly resolution
timesteps = pd.date_range('2024-01-01', periods=168, freq='h')  # 7 days
hours = np.arange(len(timesteps))
hour_of_day = hours % 24

np.random.seed(42)

# Heat demand: daily pattern
daily_pattern = np.select(
    [
        (hour_of_day >= 6) & (hour_of_day < 9),
        (hour_of_day >= 9) & (hour_of_day < 17),
        (hour_of_day >= 17) & (hour_of_day < 22),
    ],
    [80, 50, 70],
    default=30,
).astype(float)

heat_demand = daily_pattern + np.random.normal(0, 5, len(timesteps))
heat_demand = np.clip(heat_demand, 20, 100)

print(f'Timesteps: {len(timesteps)} hours ({len(timesteps) / 24:.0f} days)')
print(f'Heat demand: {heat_demand.min():.0f} - {heat_demand.max():.0f} kW')

In [ ]:
def build_system(timesteps, heat_demand):
    """Build a simple heating system with storage."""
    fs = fx.FlowSystem(timesteps)
    fs.add_elements(
        # Buses
        fx.Bus('Gas'),
        fx.Bus('Heat'),
        # Effects
        fx.Effect('costs', '€', 'Total Costs', is_standard=True, is_objective=True),
        # Gas Supply
        fx.Source(
            'GasGrid',
            outputs=[fx.Flow('Gas', bus='Gas', size=500, effects_per_flow_hour=0.05)],
        ),
        # Gas Boiler
        fx.linear_converters.Boiler(
            'Boiler',
            thermal_efficiency=0.92,
            thermal_flow=fx.Flow('Q_th', bus='Heat', size=150),
            fuel_flow=fx.Flow('Q_fuel', bus='Gas'),
        ),
        # Thermal Storage
        fx.Storage(
            'Storage',
            capacity_in_flow_hours=100,
            initial_charge_state=0,
            eta_charge=0.95,
            eta_discharge=0.95,
            relative_loss_per_hour=0.01,
            charging=fx.Flow('Charge', bus='Heat', size=50),
            discharging=fx.Flow('Discharge', bus='Heat', size=50),
        ),
        # Heat Demand
        fx.Sink(
            'HeatDemand',
            inputs=[fx.Flow('Q_th', bus='Heat', size=1, fixed_relative_profile=heat_demand)],
        ),
    )
    return fs


flow_system = build_system(timesteps, heat_demand)
print(f'System: {len(timesteps)} timesteps')

## Full Optimization (Baseline)

First, solve the full problem as a baseline:

In [ ]:
solver = fx.solvers.HighsSolver()

start = timeit.default_timer()
fs_full = flow_system.copy()
fs_full.optimize(solver)
time_full = timeit.default_timer() - start

print(f'Full optimization: {time_full:.2f} seconds')
print(f'Cost: {fs_full.solution["costs"].item():.2f} €')

## Rolling Horizon Optimization

The `optimize.rolling_horizon()` method divides the time horizon into segments that are solved sequentially:

```
Full horizon:  |-------- 168 hours ---------------------------------------...|
                
Segment 1:     |==== 24h ====|-- overlap --|
Segment 2:                   |==== 24h ====|-- overlap --|
Segment 3:                                 |==== 24h ====|-- overlap --|
...                                                  
```

Key parameters:
- **horizon**: Timesteps per segment (excluding overlap)
- **overlap**: Additional lookahead timesteps (improves storage optimization)
- **nr_of_previous_values**: Flow history transferred between segments

In [ ]:
start = timeit.default_timer()
fs_rolling = flow_system.copy()
segments = fs_rolling.optimize.rolling_horizon(
    solver,
    horizon=24,  # Daily segments
    overlap=6,  # 6-hour lookahead
)
time_rolling = timeit.default_timer() - start

print(f'Rolling horizon: {time_rolling:.2f} seconds ({len(segments)} segments)')
print(f'Cost: {fs_rolling.solution["costs"].item():.2f} €')

## Compare Results

In [ ]:
cost_full = fs_full.solution['costs'].item()
cost_rolling = fs_rolling.solution['costs'].item()
cost_gap = (cost_rolling - cost_full) / cost_full * 100

results = pd.DataFrame(
    {
        'Method': ['Full optimization', 'Rolling horizon'],
        'Time [s]': [time_full, time_rolling],
        'Cost [€]': [cost_full, cost_rolling],
        'Cost Gap [%]': [0.0, cost_gap],
    }
).set_index('Method')

results.round(2)

## Visualize: Heat Balance Comparison

In [ ]:
fs_full.statistics.plot.balance('Heat').figure.update_layout(title='Heat Balance (Full)')

In [ ]:
fs_rolling.statistics.plot.balance('Heat').figure.update_layout(title='Heat Balance (Rolling)')

## Storage State Continuity

Rolling horizon transfers storage charge states between segments to ensure continuity:

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=['Full Optimization', 'Rolling Horizon']
)

# Full optimization
charge_full = fs_full.solution['Storage|charge_state'].values[:-1]  # Drop final NaN
fig.add_trace(go.Scatter(x=timesteps, y=charge_full, name='Full', line=dict(color='blue')), row=1, col=1)

# Rolling horizon
charge_rolling = fs_rolling.solution['Storage|charge_state'].values[:-1]
fig.add_trace(go.Scatter(x=timesteps, y=charge_rolling, name='Rolling', line=dict(color='orange')), row=2, col=1)

fig.update_yaxes(title_text='Charge State [kWh]', row=1, col=1)
fig.update_yaxes(title_text='Charge State [kWh]', row=2, col=1)
fig.update_layout(height=400, showlegend=False)
fig.show()

## Inspect Individual Segments

The method returns the individual segment FlowSystems, which can be inspected:

In [ ]:
print(f'Number of segments: {len(segments)}')
print()
for i, seg in enumerate(segments):
    start_time = seg.timesteps[0]
    end_time = seg.timesteps[-1]
    cost = seg.solution['costs'].item()
    print(f'Segment {i + 1}: {start_time} → {end_time} | Cost: {cost:.2f} €')

## Effect of Overlap

The overlap parameter provides lookahead for storage optimization. Let's compare different overlap values:

In [ ]:
overlaps = [0, 3, 6, 12, 24]
overlap_results = []

for overlap in overlaps:
    fs = flow_system.copy()
    start = timeit.default_timer()
    fs.optimize.rolling_horizon(solver, horizon=24, overlap=overlap)
    elapsed = timeit.default_timer() - start
    cost = fs.solution['costs'].item()
    gap = (cost - cost_full) / cost_full * 100
    overlap_results.append({'Overlap [h]': overlap, 'Time [s]': elapsed, 'Cost [€]': cost, 'Gap [%]': gap})

pd.DataFrame(overlap_results).round(2)

## When to Use Rolling Horizon

| Use Case | Recommendation |
|----------|----------------|
| **Memory limits** | Large problems that exceed available memory |
| **Operational planning** | When limited foresight is realistic |
| **Quick approximate solutions** | Faster than full optimization |
| **Investment decisions** | Use full optimization instead |

### Limitations

- **No investments**: `InvestParameters` are not supported (raises error)
- **Suboptimal storage**: Limited foresight may miss long-term storage opportunities
- **Global constraints**: `flow_hours_max` etc. cannot be enforced globally

## API Reference

```python
segments = flow_system.optimize.rolling_horizon(
    solver,              # Solver instance
    horizon=100,         # Timesteps per segment
    overlap=0,           # Additional lookahead timesteps
    nr_of_previous_values=1,  # Flow history for uptime/downtime tracking
)

# Combined solution on original FlowSystem
flow_system.solution['costs'].item()

# Individual segment solutions
for seg in segments:
    print(seg.solution['costs'].item())
```

## Summary

You learned how to:

- Use **`optimize.rolling_horizon()`** to decompose large problems
- Choose **horizon** and **overlap** parameters
- Understand the **trade-offs** vs. full optimization

### Key Takeaways

1. **Rolling horizon** is useful for memory-limited or operational planning problems
2. **Overlap** improves solution quality at the cost of computation time
3. **Storage states** are automatically transferred between segments
4. Use **full optimization** for investment decisions